In [31]:
import os
import json
import numpy as np
import pickle 
import tqdm

# Roles, currently doesn't distinguish between carry/support in safelane
# 1 safelane
# 2 mid
# 3 offlane
# 4 jungle
# 5 <- label by looking at wards placed and gold earned


"""
Features:
GPM: rank 1-5
XPM: rank 1-5
K+A: rank 1-5
D  : rank 1-5
Last-hits: rank 1-5
Team HIDS: Binary encoded [130]


Assumption: Linear difference between ranks
Goal: Create ensemble of classifiers where 
            |C| = #heroes * #roles

"""

'\nFeatures:\nGPM: rank 1-5\nXPM: rank 1-5\nK+A: rank 1-5\nD  : rank 1-5\nLast-hits: rank 1-5\nTeam HIDS: Binary encoded [130]\n\n\nAssumption: Linear difference between ranks\nGoal: Create ensemble of classifiers where \n            |C| = #heroes * #roles\n\n'

In [60]:
def get_oh_by_rank(rank: int):
    oh = np.zeros(5)
    oh[rank] = 1
    return oh

X = []
y = []
for filename in tqdm.tqdm(os.listdir("../data/parsed")):       
    with open("../data/parsed/" + filename, "rb") as f:
        match = pickle.load(f)
        players = match["players"]

        t0 = list(filter(lambda p: p["isRadiant"], players))
        t1 = list(filter(lambda p: not p["isRadiant"], players))
        t0_hids = [p["hero_id"] for p in t0]
        t1_hids = [p["hero_id"] for p in t1]
        t0_gpm_rank = sorted([(p["hero_id"], p["gold_per_min"]) for p in t0], key=lambda x: x[1], reverse=False)
        t1_gpm_rank = sorted([(p["hero_id"], p["gold_per_min"]) for p in t1], key=lambda x: x[1], reverse=False)
        t0_xpm_rank = sorted([(p["hero_id"], p["xp_per_min"]) for p in t0], key=lambda x: x[1], reverse=False)
        t1_xpm_rank = sorted([(p["hero_id"], p["xp_per_min"]) for p in t1], key=lambda x: x[1], reverse=False)
        t0_ka_rank = sorted([(p["hero_id"], p["kills"] + p["assists"]) for p in t0], key=lambda x: x[1], reverse=False)
        t1_ka_rank = sorted([(p["hero_id"], p["kills"] + p["assists"]) for p in t1], key=lambda x: x[1], reverse=False)
        t0_d_rank = sorted([(p["hero_id"], p["deaths"]) for p in t0], key=lambda x: x[1], reverse=False)
        t1_d_rank = sorted([(p["hero_id"], p["deaths"]) for p in t1], key=lambda x: x[1], reverse=False)
        t0_lasthits_rank = sorted([(p["hero_id"], p["last_hits"]) for p in t0], key=lambda x: x[1], reverse=False)
        t1_lasthits_rank = sorted([(p["hero_id"], p["last_hits"]) for p in t1], key=lambda x: x[1], reverse=False)
        t0_wards_placed = []
        t1_wards_placed = []

        for p in t0:
            item_uses = p["item_uses"]
            c = 0
            if "ward_observer" in item_uses:
                c += item_uses["ward_observer"]
            if "ward_sentry" in item_uses:
                c += item_uses["ward_sentry"]
            t0_wards_placed.append((p["hero_id"], c))

        for p in t1:
            item_uses = p["item_uses"]
            c = 0
            if "ward_observer" in item_uses:
                c += item_uses["ward_observer"]
            if "ward_sentry" in item_uses:
                c += item_uses["ward_sentry"]
            t1_wards_placed.append((p["hero_id"], c))

        for h in t0:
            features = []
            hid = h["hero_id"]
            onehot_heroes = np.zeros((np.max(t0_hids) + 1))
            for team_hid in t0_hids:
                onehot_heroes[team_hid] = 1
            features.append(get_oh_by_rank(t0_gpm_rank.index((hid, h["gold_per_min"]))))
            features.append(get_oh_by_rank(t0_xpm_rank.index((hid, h["xp_per_min"]))))
            features.append(get_oh_by_rank(t0_ka_rank.index((hid,  h["kills"] + h["assists"]))))
            features.append(get_oh_by_rank(t0_d_rank.index((hid, h["deaths"]))))
            features.append(get_oh_by_rank(t0_lasthits_rank.index((hid, h["last_hits"]))))
            features.append(onehot_heroes)

            if hid == t0_wards_placed[0][0]:
                y.append(5)
            else:
                y.append(h["lane_role"])
            X.append(np.array(features))

        for h in t1:
            features = []
            hid = h["hero_id"]
            onehot_heroes = np.zeros((np.max(t1_hids) + 1))
            for team_hid in t1_hids:
                onehot_heroes[team_hid] = 1
            features.append(get_oh_by_rank(t1_gpm_rank.index((hid, h["gold_per_min"]))))
            features.append(get_oh_by_rank(t1_xpm_rank.index((hid, h["xp_per_min"]))))
            features.append(get_oh_by_rank(t1_ka_rank.index((hid,  h["kills"] + h["assists"]))))
            features.append(get_oh_by_rank(t1_d_rank.index((hid, h["deaths"]))))
            features.append(get_oh_by_rank(t1_lasthits_rank.index((hid, h["last_hits"]))))
            features.append(onehot_heroes)

            if hid == t1_wards_placed[0][0]:
                y.append(5)
            else:
                y.append(h["lane_role"])
            
            X.append(np.array(features))

        


100%|██████████| 50000/50000 [01:31<00:00, 544.18it/s]


In [59]:
X = np.array(X)
X.shape

(500000, 6)

In [56]:
import pandas as pd

df_pos = pd.DataFrame(y)

df_pos[0].value_counts()


1    155940
3    155206
5    100000
2     85380
4      3474
Name: 0, dtype: int64

In [57]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score,cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix


clf = DecisionTreeClassifier(random_state=0)
cross_val_score(clf, X, y, cv=10)

/home/bao/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: only size-1 arrays can be converted to Python scalars

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/home/bao/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/bao/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 890, in fit
    super().fit(
  File "/home/bao/anaconda3/lib/python3.8/site-packages/sklearn/tree/_classes.py", line 156, in fit
    X, y = self._validate_data(X, y,
  File "/home/bao/anaconda3/lib/python3.8/site-packages/sklearn/base.py", line 429, in _validate_data
    X = check_array(X, **check_X_params)
  File "/home/bao/anaconda3/lib/pyt

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan])

In [7]:
y_pred = cross_val_predict(clf, X, y, cv=10)
confusion_matrix(y, y_pred)

array([[96677,  8361, 23658,    11],
       [24356, 10056,  8104,     0],
       [39166,  3839, 34459,     3],
       [  947,    59,   302,     2]])

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score,cross_val_predict, train_test_split
from sklearn.metrics import confusion_matrix

clf = LogisticRegression(random_state=0)
y_pred = cross_val_predict(clf, X, y, cv=10)
confusion_matrix(y, y_pred)


ValueError: setting an array element with a sequence.